In [12]:
# pip install openpyxl python-pptx

In [ ]:
import openpyxl
from pptx import Presentation

def find_table(slide):
    for shape in slide.shapes:
        if shape.has_table:
            return shape.table
    return None

# Map Excel headers to (row, col) in the PowerPoint table
excel_to_table_map = {
    "PS No": (0, 1),
    "Name": (0, 3),
    "DOB | Age": (1, 1),
    "Designation": (1, 2),
    "Site / Office": (2, 1),
    "IC | Location": (2, 2),
    "DOJ": (3, 1),
    "IS Name & Grade": (3, 2),
    "Grade": (4, 1),
    "Last Promotion Date": (4, 2),
    "DOR": (5, 1),
    "FAIR 23|24|25": (5, 2),
    "CP Rating 2025": (5, 3),
    "Total Yrs. of Exp| L&T Exp": (6, 1),
    "DC Score| Year": (6, 2),
    "Qualification | Year": (7, 1),
    "Current Role": (9, 0),
    "Role Envisaged in 2025-26 (in Terms of Job Rotation/Job Enlargement)": (9, 1),
}

# Load Excel with data_only=True to get calculated values
wb = openpyxl.load_workbook('Excel-Test.xlsx', data_only=True)
ws = wb.active
excel_headers = [cell.value for cell in next(ws.iter_rows(min_row=1, max_row=1))]

# Load PowerPoint template
prs = Presentation('Role.pptx')
template_slide = prs.slides[0]
table = find_table(template_slide)
if table is None:
    raise Exception("No table found in the template slide.")

# Prepare a new presentation for output
final_prs = Presentation()
final_prs.slide_width = prs.slide_width
final_prs.slide_height = prs.slide_height

for row in ws.iter_rows(min_row=2, values_only=True):
    prs_temp = Presentation('Role.pptx')
    slide = prs_temp.slides[0]
    table = find_table(slide)
    # Fill table by mapping
    for idx, value in enumerate(row):
        header = excel_headers[idx]
        if header in excel_to_table_map:
            r, c = excel_to_table_map[header]
            table.cell(r, c).text = str(value) if value is not None else ""
    # Copy filled slide to final presentation
    blank_slide_layout = final_prs.slide_layouts[6]
    new_slide = final_prs.slides.add_slide(blank_slide_layout)
    for shape in slide.shapes:
        el = shape.element
        new_slide.shapes._spTree.insert_element_before(el, 'p:extLst')

final_prs.save('Filled_Role.pptx')